In [ ]:
import glob
import random
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
DATASET_DIR = './datasets/raw'
DATASET_IMAGES = [DATASET_DIR + '/' + image for image in glob.glob('*.jpg', root_dir=DATASET_DIR)]

In [ ]:
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.left'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['ytick.left'] = False
plt.rcParams['xtick.labelbottom'] = False
plt.rcParams['ytick.labelleft'] = False

In [ ]:
def dog_filter(image, kernel_size=0, sigma=1.4, k_sigma=1.6, gamma=1):
    """DoG(Difference of Gaussians)

    Args:
        image: OpenCV Image
        kernel_size: Gaussian Blur Kernel Size
        sigma: sigma for small Gaussian filter
        k_sigma: large/small for sigma Gaussian filter
        gamma: scale parameter for DoG signal to make sharp

    Returns:
        Image after applying the DoG.
    """
    g1 = cv.GaussianBlur(image, (kernel_size, kernel_size), sigma)
    g2 = cv.GaussianBlur(image, (kernel_size, kernel_size), sigma * k_sigma)
    return g1 - gamma * g2

def xdog_filter(image,
                *,
                kernel_size=0,
                sigma=1.4,
                k_sigma=1.6,
                epsilon=0,
                phi=10,
                gamma=0.98):
    """XDoG(Extended Difference of Gaussians)

    Args:
        image: OpenCV Image
        kernel_size: Gaussian Blur Kernel Size
        sigma: sigma for small Gaussian filter
        k_sigma: large/small for sigma Gaussian filter
        eps: threshold value between dark and bright
        phi: soft threshold
        gamma: scale parameter for DoG signal to make sharp

    Returns:
        Image after applying the XDoG.
    """
    epsilon /= 255
    dog = dog_filter(image, kernel_size, sigma, k_sigma, gamma)
    dog /= dog.max()
    e = 1 + np.tanh(phi * (dog - epsilon))
    e[e >= 1] = 1
    return e.astype('uint8') * 255


In [ ]:
def process(image_path: str, *, debug: bool = False):
    # Read image as RGB
    rgb = cv.cvtColor(cv.imread(image_path), cv.COLOR_BGR2RGB)

    # Adjust contrast
    contrast = 3.0
    brightness = 0
    rgb_adj = cv.convertScaleAbs(rgb, alpha=contrast, beta=brightness)

    # Convert to HSV
    hsv = cv.cvtColor(rgb, cv.COLOR_RGB2HSV)
    hsv_adj = cv.cvtColor(rgb_adj, cv.COLOR_RGB2HSV)

    if debug:
        plt.title('RGB and HSV')
        plt.subplot(2,2,1)
        plt.imshow(rgb)
        plt.subplot(2,2,2)
        plt.imshow(rgb_adj)
        plt.subplot(2,2,3)
        plt.imshow(hsv)
        plt.subplot(2,2,4)
        plt.imshow(hsv_adj)
        plt.show()

    # Preform XDoG
    shapes = xdog_filter(hsv_adj[:,:,1],kernel_size=9,sigma=0.5,k_sigma=1.6,epsilon=5,phi=1,gamma=0.99)

    # Remove impulse noise
    denoised = cv.medianBlur(shapes, 3)

    if debug:
        plt.title('Denoised XDoG')
        plt.imshow(denoised)
        plt.show()

    # Join blobs
    kernel = np.ones((15, 15), np.uint8)
    blobs = cv.erode(cv.dilate(denoised, kernel), kernel)

    # Threshold
    _, thresh = cv.threshold(blobs, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)

    if debug:
        plt.title('Thresholded blobs')
        plt.imshow(thresh)
        plt.show()

    # Find contours
    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    if debug:
        plt.title('Raw contours (count = %d)' % len(contours))
        mat = np.zeros_like(thresh)
        cv.drawContours(mat, contours, -1, (255), 3)
        plt.imshow(mat)
        plt.show()

    # Segmentation and contour canvases
    sgm = np.zeros_like(rgb)
    cnt = np.zeros_like(rgb)
    cnt[:,:,0] = hsv[:,:,2]
    cnt[:,:,1] = hsv[:,:,2]
    cnt[:,:,2] = hsv[:,:,2]

    for index, contour in enumerate(contours):

        # Contour mask
        hull = cv.convexHull(contour)
        contour_mask = np.zeros_like(hsv[:,:,0])
        cv.drawContours(contour_mask, [hull], -1, (255), cv.FILLED)

        # Saturation and value mask
        sv_mask = np.ones_like(rgb[:,:,0]) * 255 * ((hsv[:,:,1] > 60) & (hsv[:,:,2] > 30))

        # Combined mask
        mask = sv_mask & contour_mask

        # Skip if mask is bad
        if np.max(mask) == 0:
            # Mask is empty
            continue
        if cv.contourArea(hull) < 1000:
            # Contour is too small
            continue
        if np.any(hull[:,:,0] <= 10) or \
            np.any(hull[:,:,0] >= rgb.shape[1] - 10) or \
            np.any(hull[:,:,1] <= 10) or \
            np.any(hull[:,:,1] >= rgb.shape[0] - 10):
            # Countour touches image edge
            continue

        if debug:
            plt.title('Hulled contour and saturation masks (index = %d)' % (index + 1))
            plt.subplot(1,2,1)
            mat = rgb.copy()
            mat[contour_mask == 0] = 255
            plt.imshow(mat)
            plt.subplot(1,2,2)
            mat[sv_mask == 0] = 255
            plt.imshow(mat)
            plt.show()

        # Average hue of masked area
        colors = hsv[:,:,0] & mask
        hue = int(np.median(colors[mask != 0]))

        # Hue classes
        HUE_DIP_SWITCH = 170
        HUE_TERMINAL_BLOCK = 68
        HUE_ARDUINO = 105
        HUE_EPSILON = 30.0

        # Hue class distances
        distance_dip_switch = np.linalg.norm(hue - HUE_DIP_SWITCH)
        distance_terminal_block = np.linalg.norm(hue - HUE_TERMINAL_BLOCK)
        distance_arduino = np.linalg.norm(hue - HUE_ARDUINO)

        # Contour classification
        if distance_dip_switch < HUE_EPSILON or distance_dip_switch > 180.0 - HUE_EPSILON:
            sgm[:,:,0] |= contour_mask
            cv.drawContours(cnt, [hull], -1, (255,0,0), 5)
        elif distance_terminal_block < HUE_EPSILON:
            sgm[:,:,1] |= contour_mask
            cv.drawContours(cnt, [hull], -1, (0,255,0), 5)
        elif distance_arduino < HUE_EPSILON:
            sgm[:,:,2] |= contour_mask
            cv.drawContours(cnt, [hull], -1, (0,0,255), 5)
        else:
            # Unclassified
            pass

    return rgb, sgm, cnt

_, _, cnt = process(DATASET_IMAGES[35], debug=True)
plt.imshow(cnt)
plt.show()

In [ ]:
for image in DATASET_IMAGES[:]:

    rgb, sgm, cnt = process(image)

    # Side-by-side plot
    # plt.subplots(figsize=(15, 5))
    # plt.title(image)
    # plt.subplot(1,3,1)
    # plt.imshow(rgb)
    # plt.subplot(1,3,2)
    # plt.imshow(sgm)
    # plt.subplot(1,3,3)
    # plt.imshow(cnt)
    # plt.show()

    # Grayscale plot
    plt.title(image)
    plt.imshow(cnt)
    plt.show()